In [2]:
import torch
import torch.nn as nn


In [24]:
N, C, W = 4, 2, 3
input = torch.tensor([[[2. ,4,1], [2,1,2]], [[1,2,3], [4,1,1]], [[2,1,3],[3,2,4]], [[1,4,2], [2,4,3]] ])

layer_norm = nn.LayerNorm([C, W])
output = layer_norm(input)

input.shape , output.shape, output


(torch.Size([4, 2, 3]),
 torch.Size([4, 2, 3]),
 tensor([[[ 0.0000,  2.0000, -1.0000],
          [ 0.0000, -1.0000,  0.0000]],
 
         [[-0.8660,  0.0000,  0.8660],
          [ 1.7320, -0.8660, -0.8660]],
 
         [[-0.5222, -1.5667,  0.5222],
          [ 0.5222, -0.5222,  1.5667]],
 
         [[-1.5076,  1.2060, -0.6030],
          [-0.6030,  1.2060,  0.3015]]], grad_fn=<NativeLayerNormBackward0>))

In [27]:
batch_norm = nn.BatchNorm1d(C)

output = batch_norm(input)

input.shape , output.shape, output

batch_norm.running_mean

tensor([0.2167, 0.2417])

In [9]:
weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
embedding = nn.Embedding.from_pretrained(weight)

inputs = torch.LongTensor([0, 1])
embedding(inputs), embedding(inputs).shape

(tensor([[1.0000, 2.3000, 3.0000],
         [4.0000, 5.1000, 6.3000]]),
 torch.Size([2, 3]))

In [16]:
prob = nn.functional.softmax(weight, dim=-1)

prob, torch.log(prob)

(tensor([[0.0829, 0.3043, 0.6128],
         [0.0715, 0.2149, 0.7135]]),
 tensor([[-2.4898, -1.1898, -0.4898],
         [-2.6375, -1.5375, -0.3375]]))

In [25]:
classifier = nn.Sequential(
            nn.Linear(2, 4),
            nn.ReLU(),
            nn.Linear(4, 6, bias=False),
        )


input = torch.FloatTensor([[[1, 2], [1, 2]], [[1, 2], [1, 2]], [[1, 2], [1, 2]]])
input.shape, classifier(input).shape

(torch.Size([3, 2, 2]), torch.Size([3, 2, 6]))

In [83]:
targets = torch.LongTensor([[1, 2, 3, 3, 1], [1, 0, 1, 2, 3], [3, 1, 0, 0, 0]])
# targets = torch.LongTensor([4, 2, 3])
targets_onehot = torch.LongTensor(targets.shape + (4, )).zero_()
targets_onehot.scatter_(-1, targets.unsqueeze(-1).long(), 1)
targets_onehot, targets_onehot.shape


(tensor([[[0, 1, 0, 0],
          [0, 0, 1, 0],
          [0, 0, 0, 1],
          [0, 0, 0, 1],
          [0, 1, 0, 0]],
 
         [[0, 1, 0, 0],
          [1, 0, 0, 0],
          [0, 1, 0, 0],
          [0, 0, 1, 0],
          [0, 0, 0, 1]],
 
         [[0, 0, 0, 1],
          [0, 1, 0, 0],
          [1, 0, 0, 0],
          [1, 0, 0, 0],
          [1, 0, 0, 0]]]),
 torch.Size([3, 5, 4]))

In [87]:
log_probas = torch.log(torch.softmax(torch.FloatTensor([
    [[4, 2, 3, 0], [1, 2, 1, 0], [3, 2, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
    [[-4, -2, -3, 0], [1, -2, 1, 0], [-3, 2, 0, 0], [1, 1, 0, 0], [0, 0, 0, 0]],
    [[1, 1, 1, 0], [1, 2, 3, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
]), dim=-1))
mask = torch.LongTensor([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0]])



T = torch.sum(mask, dim=-1) 
loss = torch.sum(log_probas * targets_onehot, dim=(1, 2))/T

log_probas, mask, T, log_probas.shape, -loss.mean()

(tensor([[[-0.4197, -2.4197, -1.4197, -4.4197],
          [-1.6265, -0.6265, -1.6265, -2.6265],
          [-0.3835, -1.3835, -3.3835, -3.3835],
          [-1.3863, -1.3863, -1.3863, -1.3863],
          [-1.3863, -1.3863, -1.3863, -1.3863]],
 
         [[-4.1852, -2.1852, -3.1852, -0.1852],
          [-0.8828, -3.8828, -0.8828, -1.8828],
          [-5.2448, -0.2448, -2.2448, -2.2448],
          [-1.0064, -1.0064, -2.0064, -2.0064],
          [-1.3863, -1.3863, -1.3863, -1.3863]],
 
         [[-1.2143, -1.2143, -1.2143, -2.2143],
          [-2.4402, -1.4402, -0.4402, -3.4402],
          [-1.3863, -1.3863, -1.3863, -1.3863],
          [-1.3863, -1.3863, -1.3863, -1.3863],
          [-1.3863, -1.3863, -1.3863, -1.3863]]]),
 tensor([[1, 1, 1, 0, 0],
         [1, 1, 1, 1, 0],
         [1, 1, 0, 0, 0]]),
 tensor([3, 4, 2]),
 torch.Size([3, 5, 4]),
 tensor(2.9946))

In [29]:
(4, 5) + (1, )

(4, 5, 1)

In [150]:
def split_heads(tensor, num_heads=1):
    batch_size, seq_length, heads_dim  = tensor.shape
    dim = heads_dim // num_heads
    output = torch.reshape(tensor, ((batch_size, seq_length, num_heads, dim)))
    output.transpose_(1, 2)
    return output

def merge_heads(tensor):
    batch_size, num_heads, seq_length, dim  = tensor.shape
    output = torch.transpose(tensor, 1 ,2)
    output = torch.reshape(output, (batch_size, seq_length, num_heads * dim))
    return output

tensor = torch.FloatTensor([[[1, 2]]])
tensor = torch.FloatTensor([
    [[-1, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11], [0, 0, 0, 0, 0, 0]], 
    [[12, 13, 14, 15, 16, 17], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
])
num_heads = 2
out = split_heads(tensor, num_heads)
# out = tensor.reshape(*tensor.shape[:2], num_heads, -1).permute(0, 2, 1, 3)
# tin = merge_heads(out)
# tensor.shape, out.shape, out, tin.shape, tin, tin == tensor
b, seq, word, head = 0, 1, 2, 1
out[b, head, seq,  word], tensor[b, seq, num_heads*head+word], out



(tensor(11.),
 tensor(10.),
 tensor([[[[-1.,  1.,  2.],
           [ 6.,  7.,  8.],
           [ 0.,  0.,  0.]],
 
          [[ 3.,  4.,  5.],
           [ 9., 10., 11.],
           [ 0.,  0.,  0.]]],
 
 
         [[[12., 13., 14.],
           [ 0.,  0.,  0.],
           [ 0.,  0.,  0.]],
 
          [[15., 16., 17.],
           [ 0.,  0.,  0.],
           [ 0.,  0.,  0.]]]]))

In [145]:
seq_length = 4
prev_masks = torch.tril(torch.ones(seq_length, seq_length))
next_masks = -1e4*torch.triu(torch.ones(seq_length, seq_length), diagonal=1)
mask = prev_masks + next_masks

mask, torch.softmax(mask, dim=1)

(tensor([[ 1.0000e+00, -1.0000e+04, -1.0000e+04, -1.0000e+04],
         [ 1.0000e+00,  1.0000e+00, -1.0000e+04, -1.0000e+04],
         [ 1.0000e+00,  1.0000e+00,  1.0000e+00, -1.0000e+04],
         [ 1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00]]),
 tensor([[1.0000, 0.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500]]))

In [148]:
input = torch.FloatTensor([[1, 2, 3, 4], [4, 3, 0, 0], [1, 0, 0, 0]])
torch.arange(1, 5)
positions = torch.zeros(input.shape).to(int) + torch.arange(4)
positions

tensor([[0, 1, 2, 3],
        [0, 1, 2, 3],
        [0, 1, 2, 3]])

In [154]:
1e4 == 10**4

True

In [7]:
def var_mean_batch(data):
    "data of shape (batch_size, channels, features)"
    return torch.var_mean(data, dim=0, keepdim=True)

data = torch.FloatTensor([
    [[2, 4, 3], [2, 1, 2]],
    [[1, 2, 3], [4, 1, 1]],
    [[2, 1, 3], [3, 2, 4]],
    [[1, 4, 2], [2, 4, 3]]
    ])
data1 = torch.FloatTensor([
    [[4, 3, 2], [1, 4, 3]],
    [[3, 4, 1], [4, 2, 2]],
    [[3, 2, 3], [4, 1, 2]],
    [[1, 1, 3], [4, 1, 2]]
])
data = data1
var_b, mu_b = torch.var_mean(data, dim=(0, 2), keepdim=True, unbiased=False)
var_l, mu_l = torch.var_mean(data, dim=(1, 2), keepdim=True, unbiased=False)
var_i, mu_i = torch.var_mean(data, dim=2, keepdim=True, unbiased=False)

print (mu_b, var_b)
print (mu_l, var_l)
print (mu_i, var_i)
data.shape, mu_b.shape, mu_l.shape, mu_i.shape

tensor([[[2.5000],
         [2.5000]]]) tensor([[[1.0833],
         [1.4167]]])
tensor([[[2.8333]],

        [[2.6667]],

        [[2.5000]],

        [[2.0000]]]) tensor([[[1.1389]],

        [[1.2222]],

        [[0.9167]],

        [[1.3333]]])
tensor([[[3.0000],
         [2.6667]],

        [[2.6667],
         [2.6667]],

        [[2.6667],
         [2.3333]],

        [[1.6667],
         [2.3333]]]) tensor([[[0.6667],
         [1.5556]],

        [[1.5556],
         [0.8889]],

        [[0.2222],
         [1.5556]],

        [[0.8889],
         [1.5556]]])


(torch.Size([4, 2, 3]),
 torch.Size([1, 2, 1]),
 torch.Size([4, 1, 1]),
 torch.Size([4, 2, 1]))